## Computing PCA using RDDs

##  PCA

The vectors that we want to analyze have length, or dimension, of 365, corresponding to the number of 
days in a year.

We will perform [Principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis)
on these vectors. There are two steps to this process:

1. Computing the covariance matrix: this is a  simple computation. However, it takes a long time to compute and it benefits from using an RDD because it involves all of the input vectors.
2. Computing the eigenvector decomposition. this is a more complex computation, but it takes a fraction of a second because the size to the covariance matrix is $365 \times 365$, which is quite small. We do it on the head node usin `linalg`

### Computing the covariance matrix
Suppose that the data vectors are the column vectors denoted $x$ then the covariance matrix is defined to be
$$
E(x x^T)-E(x)E(x)^T
$$

Where $x x^T$ is the **outer product** of $x$ with itself.

If the data that we have is $x_1,x_2,x_n$ then the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

### The effect of  `nan`s in arithmetic operations
* We use an RDD of numpy arrays, instead of Dataframes.
* Why? Because numpy treats `nan` entries correctly:
  * In numpy `5+nan=5` while in dataframes `5+nan=nan`

### Performing Cov matrix on vectors with NaNs
As it happens, we often get vectors $x$ in which some, but not all, of the entries are `nan`. 
Suppose that we want to compute the mean of the elements of $x$. If we use `np.mean` we will get the result `nan`. A useful alternative is to use `np.nanmean` which removes the `nan` elements and takes the mean of the rest.

In [1]:
import numpy as np
X=np.array([1,1,1,2])
print('mean of',X,'=',np.mean(X))
print('nanmean of',X,'=',np.nanmean(X))
X=np.array([1,1,np.NaN,2])
print('mean of',X,'=',np.mean(X))
print('nanmean of',X,'=',np.nanmean(X))

mean of [1 1 1 2] = 1.25
nanmean of [1 1 1 2] = 1.25
mean of [  1.   1.  nan   2.] = nan
nanmean of [  1.   1.  nan   2.] = 1.33333333333


#### When should you not use `np.nanmean` ?
Using `n.nanmean` is equivalent to assuming that choice of which elements to remove is independent of the values of the elements. 
* Example of bad case: suppose the larger elements have a higher probability of being `nan`. In that case `np.nanmean` will under-estimate the mean

#### Computing the covariance  when there are `nan`s
The covariance is a mean of outer products.

If the data that we have is $x_1,x_2,x_n$ then the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

In [24]:
#import findspark
#findspark.init()
from pyspark import SparkContext,SparkConf

def create_sc(pyFiles):
    sc_conf = SparkConf()
    sc_conf.setAppName("Weather_PCA")
    sc_conf.set('spark.executor.memory', '2g')
    sc_conf.set('spark.executor.cores', '4')
    sc_conf.set('spark.cores.max', '40')
    sc_conf.set('spark.default.parallelism','10')
    sc_conf.set('spark.logConf', True)
    print(sc_conf.getAll())

    sc = SparkContext(conf=sc_conf,pyFiles=pyFiles)

    return sc 

#sc.stop()
#sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])
sc = create_sc(pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStatistics.py'])

[('spark.eventLog.enabled', 'true'), ('spark.driver.extraLibraryPath', '/usr/lib/hadoop/lib/native:/usr/lib/hadoop-lzo/lib/native'), ('spark.blacklist.decommissioning.timeout', '1h'), ('spark.app.name', 'Weather_PCA'), ('spark.yarn.appMasterEnv.SPARK_PUBLIC_DNS', '$(hostname -f)'), ('spark.executor.cores', '4'), ('spark.cores.max', '40'), ('spark.yarn.historyServer.address', 'ip-10-129-226-48.ec2.internal:18080'), ('spark.executor.extraJavaOptions', "-verbose:gc -XX:+PrintGCDetails -XX:+PrintGCDateStamps -XX:+UseConcMarkSweepGC -XX:CMSInitiatingOccupancyFraction=70 -XX:MaxHeapFreeRatio=70 -XX:+CMSClassUnloadingEnabled -XX:OnOutOfMemoryError='kill -9 %p'"), ('spark.default.parallelism', '10'), ('spark.eventLog.dir', 'hdfs:///var/log/spark/apps'), ('spark.sql.hive.metastore.sharedPrefixes', 'com.amazonaws.services.dynamodbv2'), ('spark.sql.warehouse.dir', 'hdfs:///user/spark/warehouse'), ('spark.submit.deployMode', 'client'), ('spark.history.fs.logDirectory', 'hdfs:///var/log/spark/apps'

In [25]:
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [26]:
#import sys
#sys.path.append('./lib')

import numpy as np
from lib.computeStatistics import *

### Climate data

The data we will use here comes from [NOAA](https://www.ncdc.noaa.gov/). Specifically, it was downloaded from This [FTP site](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/).

There is a large variety of measurements from all over the world, from 1870 will 2012.
in the directory `../../Data/Weather` you will find the following useful files:

* data-source.txt: the source of the data
* ghcnd-readme.txt: A description of the content and format of the data
* ghcnd-stations.txt: A table describing the Meteorological stations.



### Data cleaning

* Most measurements exists only for a tiny fraction of the stations and years. We therefor restrict our use to the following measurements:
```python
['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']
```

* 8 We consider only measurement-years that have at most 50 `NaN` entries

* We consider only measurements in the continential USA

* We partition the stations into the states of the continental USA (plus a few stations from states in canada and mexico).

In [27]:
state='NY'
data_dir='../../Data/Weather'
tarname=state+'.tgz'
parquet=state+'.parquet'

In [28]:
!rm -rf $data_dir/$tarname

command="curl https://mas-dse-open.s3.amazonaws.com/Weather/by_state/%s > %s/%s"%(tarname,data_dir,tarname)
print(command)
!$command
!ls -lh $data_dir/$tarname

curl https://mas-dse-open.s3.amazonaws.com/Weather/by_state/NY.tgz > ../../Data/Weather/NY.tgz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22.1M  100 22.1M    0     0  7546k      0  0:00:03  0:00:03 --:--:-- 6893k
-rw-rw-r-- 1 hadoop hadoop 23M Mar 26 18:47 ../../Data/Weather/NY.tgz


In [29]:
cur_dir,=!pwd
%cd $data_dir
!tar -xzf $tarname
!du ./$parquet
%cd $cur_dir

/mnt/workspace/edX-Micro-Master-in-Data-Science/big-data-analytics-using-spark/notebooks/Data/Weather
28244	./NY.parquet
/mnt/workspace/edX-Micro-Master-in-Data-Science/big-data-analytics-using-spark/notebooks/Section2-PCA/PCA


In [38]:
data_dir

'../../Data/Weather'

In [39]:
!mv ../../Data/Weather/NY.parquet /mnt/workspace/Data/NY.parquet

In [41]:
!aws s3 cp --recursive --quiet /mnt/workspace/Data/NY.parquet s3://dse-weather/NY.parquet

In [42]:
!aws s3 ls s3://dse-weather/

                           PRE 256_STAT/
                           PRE NY.parquet/
                           PRE US_stations.parquet/
                           PRE US_weather.parquet/
                           PRE info/
                           PRE weather.parquet/
2016-02-09 04:09:49 1511481989 ALL.csv.gz
2018-03-16 04:15:54        600 PrivateBootstrap.sh
2018-03-16 04:15:55        658 RunFromTerminal.sh
2018-03-16 04:15:53        300 s3hook.sh


In [7]:
local_path=data_dir+'/'+parquet
hdfs_path='/weather/'+parquet
local_path,hdfs_path

('../../Data/Weather/NY.parquet', '/weather/NY.parquet')

In [8]:
!hdfs dfs -copyFromLocal $local_path $hdfs_path

copyFromLocal: `/weather/NY.parquet/NY.parquet/_SUCCESS': File exists
copyFromLocal: `/weather/NY.parquet/NY.parquet/part-00000-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet': File exists
copyFromLocal: `/weather/NY.parquet/NY.parquet/part-00001-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet': File exists
copyFromLocal: `/weather/NY.parquet/NY.parquet/part-00002-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet': File exists
copyFromLocal: `/weather/NY.parquet/NY.parquet/part-00003-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet': File exists
copyFromLocal: `/weather/NY.parquet/NY.parquet/part-00004-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet': File exists
copyFromLocal: `/weather/NY.parquet/NY.parquet/part-00005-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet': File exists
copyFromLocal: `/weather/NY.parquet/NY.parquet/part-00006-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet': File exists
copyFromLocal: `/weather/NY

In [9]:
!hdfs dfs -du /weather/

57771504    /weather/NY.parquet
847798      /weather/US_stations.parquet
2027712578  /weather/US_weather.parquet


In [10]:
%%time
df=sqlContext.read.parquet(hdfs_path)
print(df.count())
df.show(5)

84199
+-----------+-----------+----+--------------------+-----+
|    Station|Measurement|Year|              Values|State|
+-----------+-----------+----+--------------------+-----+
|USC00303452|       PRCP|1903|[00 7E 00 7E 00 7...|   NY|
|USC00303452|       PRCP|1904|[00 00 28 5B 00 0...|   NY|
|USC00303452|       PRCP|1905|[00 00 60 56 60 5...|   NY|
|USC00303452|       PRCP|1906|[00 00 00 00 00 0...|   NY|
|USC00303452|       PRCP|1907|[00 00 00 00 60 5...|   NY|
+-----------+-----------+----+--------------------+-----+
only showing top 5 rows

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 11.4 s


In [11]:
from time import time
t=time()

N=sc.defaultParallelism
print('Number of executors=',N)
print('took',time()-t,'seconds')

Number of executors= 10
took 0.0009517669677734375 seconds


In [12]:
data_dir='/mnt/workspace/Data/'


In [15]:
%%time
from pickle import dump

state='NY'
parquet=state+'.parquet'
df=sqlContext.read.parquet('/weather/'+parquet).cache()
print(state,df.count())

NY 84199
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.16 s


In [16]:
%%time
df.show(4)

+-----------+-----------+----+--------------------+-----+
|    Station|Measurement|Year|              Values|State|
+-----------+-----------+----+--------------------+-----+
|USC00303452|       PRCP|1903|[00 7E 00 7E 00 7...|   NY|
|USC00303452|       PRCP|1904|[00 00 28 5B 00 0...|   NY|
|USC00303452|       PRCP|1905|[00 00 60 56 60 5...|   NY|
|USC00303452|       PRCP|1906|[00 00 00 00 00 0...|   NY|
+-----------+-----------+----+--------------------+-----+
only showing top 4 rows

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 154 ms


In [19]:
# %load lib/computeStatistics.py

from numpy import linalg as LA
import numpy as np

from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from time import time

def computeStatistics(sqlContext,df):
    """Compute all of the statistics for a given dataframe
    Input: sqlContext: to perform SQL queries
            df: dataframe with the fields 
            Station(string), Measurement(string), Year(integer), Values (byteArray with 365 float16 numbers)
    returns: STAT, a dictionary of dictionaries. First key is measurement, 
             second keys described in computeStats.STAT_Descriptions
    """

    sqlContext.registerDataFrameAsTable(df,'weather')
    STAT={}  # dictionary storing the statistics for each measurement
    measurements=['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']
    
    for meas in measurements:
        t=time()
        Query="SELECT * FROM weather\n\tWHERE measurement = '%s'"%(meas)
        mdf = sqlContext.sql(Query)
        print(meas,': shape of mdf is ',mdf.count())

        data=mdf.rdd.map(lambda row: unpackArray(row['Values'],np.float16))

        #Compute basic statistics
        STAT[meas]=computeOverAllDist(data)   # Compute the statistics 

        # compute covariance matrix
        OUT=computeCov(data)

        #find PCA decomposition
        eigval,eigvec=LA.eig(OUT['Cov'])

        # collect all of the statistics in STAT[meas]
        STAT[meas]['eigval']=eigval
        STAT[meas]['eigvec']=eigvec
        STAT[meas].update(OUT)

        print('time for',meas,'is',time()-t)
    
    return STAT

# Compute the overall distribution of values and the distribution of the number of nan per year
def find_percentiles(SortedVals,percentile):
    L=int(len(SortedVals)/percentile)
    return SortedVals[L],SortedVals[-L]
  
def computeOverAllDist(rdd0):
    UnDef=np.array(rdd0.map(lambda row:sum(np.isnan(row))).sample(False,0.01).collect())
    flat=rdd0.flatMap(lambda v:list(v)).filter(lambda x: not np.isnan(x)).cache()
    count,S1,S2=flat.map(lambda x: np.float64([1,x,x**2]))\
                  .reduce(lambda x,y: x+y)
    mean=S1/count
    std=np.sqrt(S2/count-mean**2)
    Vals=flat.sample(False,0.0001).collect()
    SortedVals=np.array(sorted(Vals))
    low100,high100=find_percentiles(SortedVals,100)
    low1000,high1000=find_percentiles(SortedVals,1000)
    return {'UnDef':UnDef,\
          'mean':mean,\
          'std':std,\
          'SortedVals':SortedVals,\
          'low100':low100,\
          'high100':high100,\
          'low1000':low100,\
          'high1000':high1000
          }

# description of data returned by computeOverAllDist
STAT_Descriptions=[
('SortedVals', 'Sample of values', 'vector whose length varies between measurements'),
 ('UnDef', 'sample of number of undefs per row', 'vector whose length varies between measurements'),
 ('mean', 'mean value', ()),
 ('std', 'std', ()),
 ('low100', 'bottom 1%', ()),
 ('high100', 'top 1%', ()),
 ('low1000', 'bottom 0.1%', ()),
 ('high1000', 'top 0.1%', ()),
 ('E', 'Sum of values per day', (365,)),
 ('NE', 'count of values per day', (365,)),
 ('Mean', 'E/NE', (365,)),
 ('O', 'Sum of outer products', (365, 365)),
 ('NO', 'counts for outer products', (365, 365)),
 ('Cov', 'O/NO', (365, 365)),
 ('Var', 'The variance per day = diagonal of Cov', (365,)),
 ('eigval', 'PCA eigen-values', (365,)),
 ('eigvec', 'PCA eigen-vectors', (365, 365))
]




In [23]:
sc.stop()

In [22]:
!echo $PYSPARK_PYTHON, $PYSPARK_DRIVER_PYTHON

,


In [20]:
%%time
STAT=computeStatistics(sqlContext,df)
filename=data_dir+'STAT_%s.pickle'%state
!ls -l $data_dir
dump((STAT,STAT_Descriptions),open(filename,'wb'))

TMAX : shape of mdf is  13437


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 17.0 failed 4 times, most recent failure: Lost task 0.3 in stage 17.0 (TID 87, ip-10-129-224-13.ec2.internal, executor 3): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/mnt/yarn/usercache/hadoop/appcache/application_1522080152359_0009/container_1522080152359_0009_01_000004/pyspark.zip/pyspark/worker.py", line 123, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 2.7 than that in driver 3.6, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.apache.spark.api.python.PythonRunner$$anon$1.<init>(PythonRDD.scala:234)
	at org.apach

In [9]:
X=STAT['TMAX']['Var']
for key in STAT.keys():
    Y=STAT[key]['Var']
    print(key,sum(abs(X-Y)))

TMAX 0.0
SNOW 852107.705867
SNWD 4464167.85212
TMIN 319734.5315
PRCP 1184305.12284
TOBS 277719.008938


In [11]:
!ls -l ../../Data/Weather/STAT*

-rw-r--r--  1 yoavfreund  staff  25684434 Mar 18 20:30 ../../Data/Weather/STAT_NY.pickle
-rw-r--r--  1 yoavfreund  staff  17545930 Mar 18 17:38 ../../Data/Weather/STAT_NY.pickle.gz
-rw-r--r--  1 yoavfreund  staff  26741490 Mar 18 20:19 ../../Data/Weather/STAT_RI.pickle
-rw-r--r--  1 yoavfreund  staff  13522496 Mar 10 12:06 ../../Data/Weather/STAT_SSSSBBBB.pickle.gz


In [12]:
!gzip -f ../../Data/Weather/STAT*.pickle
!ls -l ../../Data/Weather/STAT*

-rw-r--r--  1 yoavfreund  staff  14948048 Mar 18 20:30 ../../Data/Weather/STAT_NY.pickle.gz
-rw-r--r--  1 yoavfreund  staff  11717259 Mar 18 20:19 ../../Data/Weather/STAT_RI.pickle.gz
-rw-r--r--  1 yoavfreund  staff  13522496 Mar 10 12:06 ../../Data/Weather/STAT_SSSSBBBB.pickle.gz


In [13]:
for state in ['RI','NY']:
    command="aws s3  cp ../../Data/Weather/STAT_%s.pickle.gz s3://mas-dse-open/Weather/by_state/STAT_%s.pickle.gz"%(state,state)
    print(command)
    !$command

aws s3  cp ../../Data/Weather/STAT_RI.pickle.gz s3://mas-dse-open/Weather/by_state/STAT_RI.pickle.gz
upload: ../../Data/Weather/STAT_RI.pickle.gz to s3://mas-dse-open/Weather/by_state/STAT_RI.pickle.gz
aws s3  cp ../../Data/Weather/STAT_NY.pickle.gz s3://mas-dse-open/Weather/by_state/STAT_NY.pickle.gz
upload: ../../Data/Weather/STAT_NY.pickle.gz to s3://mas-dse-open/Weather/by_state/STAT_NY.pickle.gz


In [14]:
!aws s3  ls s3://mas-dse-open/Weather/by_state/ | grep STAT

2018-03-18 20:34:12   14948048 STAT_NY.pickle.gz
2018-03-18 20:33:54   11717259 STAT_RI.pickle.gz
